# Tabular classification
- Supervised credit-g classification

In [ ]:

import torch.nn
import torch.optim

import openml
import openml_pytorch
import openml_pytorch.layers
import openml_pytorch.config
import logging


############################################################################
# Enable logging in order to observe the progress while running the example.
openml.config.logger.setLevel(logging.DEBUG)
openml_pytorch.config.logger.setLevel(logging.DEBUG)
############################################################################

In [ ]:
from openml_pytorch.trainer import OpenMLTrainerModule
from openml_pytorch.trainer import OpenMLDataModule
from openml_pytorch.trainer import Callback

# Define the Model

In [ ]:
class TabularClassificationmodel(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(TabularClassificationmodel, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, output_size)
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [ ]:
model = TabularClassificationmodel(20, 2)

In [ ]:
# supervised credit-g classification
task = openml.tasks.get_task(31)

## Configure the Data Module
- Make sure the `target_col` is correctly set.

In [ ]:
data_module = OpenMLDataModule(
    type_of_data="dataframe",
    target_column="class",
    target_mode="categorical",
)

## Configure the Trainer Module

In [ ]:

trainer = OpenMLTrainerModule(
    data_module=data_module,
    verbose = True,
    epoch_count = 5,
)
openml_pytorch.config.trainer = trainer

## Run the model on the task

In [ ]:
run = openml.runs.run_model_on_task(model, task, avoid_duplicate_runs=False)

In [ ]:
run.publish()

In [ ]:
# openml.config.apikey = ''

## View loss

In [ ]:
trainer.runner.cbs[1].plot_loss()

## View learning rate

In [ ]:
trainer.runner.cbs[1].plot_lr()